# Parse data from ebike datalogger

In [ ]:
!pip3 install autokeras

In [ ]:
!pip install lightgbm

In [ ]:
!pip install optuna

# Load all the functions

In [ ]:
import random
import numpy as np
import pandas as pd
#import tensorflow as tf
#import autokeras as ak
import gc

import secrets


import xgboost as xgb
from xgboost import plot_importance, plot_tree, to_graphviz
from sklearn.datasets import load_boston

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, accuracy_score

import glob 
import os
import scipy as sp
import scipy.signal as sg


from butter_filter import signal_filter
from gen_plots import display_interesting_variables, display_all_variables
from Battery_Kalman.soc_estimator import SocEstimator
from Battery_Kalman.battery import Battery

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'browser'


from sklearn.preprocessing import StandardScaler


import geopandas as gpd
from convertbng.util import convert_bng, convert_lonlat
from sklearn.neighbors import BallTree
sns.set_theme()

# DANGEROUS DONT DO
pd.options.mode.chained_assignment = None  # default='warn'



#px.set_mapbox_access_token(open(".mapbox_token").read())

BATTERY_ENERGY_CAPACITY = 752.4 # Kilo Joules

#raw_data_path = "D:/OneDrive - Imperial College London/University Storage/Masters project/data_storage/"
raw_data_path = "/home/medad/Downloads/MastersProject/Bike_logger/Data_analysis/data_storage/data_storage/"

# Number of PAS magnets
N_PAS_MAGNETS = 12

# Pressure (mbar) at sea level where the readings are being taken.  
qnh=1032.57

ROAD_FEATURES_PKL_FP = "road_features.pkl"

def load_road_features_from_raw():

    # Grid references for London: https://getoutside.ordnancesurvey.co.uk/guides/beginners-guide-to-grid-references/
    # Read SHx files
    files = [
    "SP_RoadNode.shx",
    "TL_RoadNode.shx",
    "SU_RoadNode.shx",
    "TQ_RoadNode.shx",
     #"NC_RoadNode.shx"

    ]

    path = raw_data_path+"oproad_essh_gb/data/"

    paths = [path+file for file in files]

    road_features_geo_df = pd.concat(map(gpd.read_file, paths))

    road_features_geo_df['Longitude (°)'], road_features_geo_df['Latitude (°)'] = convert_lonlat(road_features_geo_df.geometry.x, road_features_geo_df.geometry.y)

    road_features_geo_df.to_pickle(ROAD_FEATURES_PKL_FP)
    
    return road_features_geo_df
    

def get_road_features(reload=False):
    
    if reload:
        road_features_geo_df = load_road_features_from_raw()
    else:
        road_features_geo_df = pd.read_pickle(ROAD_FEATURES_PKL_FP)
    
    return road_features_geo_df

# read raw data
def read_file(filepath):
    my_cols = range(19)

    date_parser=lambda x: pd.to_datetime(x, errors="coerce", format = "%Y-%m-%dT%H:%M:%S.%fZ", utc=True)

    
    df = pd.read_csv(filepath,
                names=my_cols,
                engine='c',
                parse_dates=[0],
                date_parser=date_parser)


    df.rename(columns={0: 'Datetime (UTC)',
                           1: 'sensor',
                          }, inplace=True)    
    
    df.dropna(inplace=True, subset=['Datetime (UTC)'])
    
    df.sort_values(by='Datetime (UTC)',inplace = True)


    df = df[~(df['Datetime (UTC)'] < '2020-03-12 18:46:00')]
    
        
    return df


def filter_df_signal(df, input_name, output_name, highcut_f):
    df[output_name] = signal_filter(df[input_name], highcut=highcut_f, method='butterworth_ba', order=5)
    return df


def energy_from_power_time(datetime_series, power_series):
    """
    Return power in kilo joules
    """
    max_seconds = 1
    
    time_delta = datetime_series.diff().dt.total_seconds().fillna(0)
    energy = power_series*time_delta
    
    energy = energy[time_delta < max_seconds]
    
    return energy.sum()/1000000

def pulse_width_pas_to_rpm(pulse_width):   
    return (1000000/pulse_width/N_PAS_MAGNETS)*60

def pulse_width_to_rpm(pulse_width):   
    return (1000000/pulse_width)*60

def get_altitude(pressure,temperature):
    altitude = ((pow((qnh / pressure), (1.0 / 5.257)) - 1) * (temperature + 273.15)) / 0.0065
    # The temperature (°C) should be the outdoor temperature (°C). 
    # Use the manual_temperature (°C) variable if temperature (°C) adjustments are required.
    return altitude

def insert_time(row):
    return row['Datetime (UTC)'].replace(minute=int(row['minute']),second=int(row['second']),microsecond=int(row['millisecond']*1000))

def process_gps(df):
    mask = df["sensor"] == 'gps'
    df_gps = df[mask]

    df_gps.rename(columns={2: 'hour',
                           3: 'minute',
                           4: 'second',
                           5: 'millisecond',
                           6: 'Latitude (°)',
                           7: 'Longitude (°)',
                           8: 'GPS altitude (m)',
                           9: 'GPS Horizontal Speed (km/h)',
                           10: 'sats',
                           11: 'gnssFixOK',
                           12: 'fix_type',
                           13: 'vehicle_heading',
                           14: 'horizontal_accuracy', # Horizontal accuracy estimate: mm
                           15: 'vertical_accuracy',   #  Vertical accuracy estimate: mm
                           16: 'speed_accuracy',     # Speed accuracy estimate: mm/s
                           17: 'heading_accuracy'    # Heading accuracy estimate (both motion and vehicle): deg
                          }, inplace=True)

    
    df_gps['Datetime (UTC)'] = df_gps.apply(lambda r: insert_time(r), axis=1)
    df_gps.sort_values(by='Datetime (UTC)',inplace = True)
    
    df_gps = df_gps[df_gps['gnssFixOK'] == 1]

    
    # There is a wierd time offset between GPS readings and all other sensor timestamps. It 
    # has been determined to be 9.5 seconds by hand adjustment. Not machine aligned.
    offset = 9.5 # seconds
    df_gps['Datetime (UTC)'] = df_gps['Datetime (UTC)'] - pd.Timedelta(offset, unit='s')

    

    time_delta = df_gps['Datetime (UTC)'].diff().dt.total_seconds().fillna(0)
    df_gps['GPS Horizontal Acceleration (km/h^2)'] = df_gps['GPS Horizontal Speed (km/h)'].diff()/time_delta
    
    
    x = df_gps['Longitude (°)'].diff().fillna(0)
    y = df_gps['Latitude (°)'].diff().fillna(0)
    
    x = signal_filter(x, highcut=100, method='butterworth_ba', order=5)
    y = signal_filter(y, highcut=100, method='butterworth_ba', order=5)

    phi, df_gps['heading (radians)'] = cart2pol(x, y)
    
    
    

    # Create a BallTree 
    tree = BallTree(road_features_geo_df[['Longitude (°)', 'Latitude (°)']].values, leaf_size=2)

    # Query the BallTree on each feature from 'appart' to find the distance
    # to the nearest 'pharma' and its id
    df_gps['Distance to nearest road feature (°)'], df_gps['id_nearest'] = tree.query(
        df_gps[['Longitude (°)', 'Latitude (°)']].values, # The input array for the query
        k=1, # The number of nearest neighbors
    )

    
    add_arc_radius_to_gps(df_gps,x,y)
    
    

    df_gps.dropna(axis=1, how='all',inplace=True)
    df_gps.head()
    
    return df_gps


def add_arc_radius_to_gps(df_gps, x, y):
        
    r,h,k = findCircle(
        np.roll(x, -1), np.roll(y, -1),
        x, y,
        np.roll(x, 1), np.roll(y, 1)
    )
    
    #print(r,h,k)
    
    r = pd.Series(r)
    r.index = df_gps.index

    
    
    df_gps['Road Curvature (m radius)'] = r


# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):

    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    
    
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a)) * 1000

def get_horizontal_distance(df):
    return haversine(df['Latitude (°)'].shift(1), df['Longitude (°)'].shift(1), df['Latitude (°)'], df['Longitude (°)'])


def add_slope_to_gps(df_gps, df_baro):
    
    
    df_baro_gps = pd.merge_asof(df_gps, df_baro, on = 'Datetime (UTC)', direction = 'forward')

    df_baro_gps['Barometric Altitude Filtered (m)'] = signal_filter(df_baro_gps['Barometric Altitude (m)'], highcut=10, method='butterworth_ba', order=2)
    
    delta_altitude = df_baro_gps['Barometric Altitude Filtered (m)'].diff()
    
    cols = ['Longitude (°)','Latitude (°)']    
    
    delta_horizontal_distance =  pd.Series(get_horizontal_distance(df_gps))
    print(delta_horizontal_distance.describe())
    
    delta_horizontal_distance.replace([np.nan], 0, inplace=True)

    delta_horizontal_distance_filtered = pd.Series(signal_filter(delta_horizontal_distance, highcut=100, method='butterworth_ba', order=2))

    slope = 100 * delta_altitude/delta_horizontal_distance_filtered
    
    slope.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    slope.index = df_gps.index
    df_gps.loc[:, 'Road Grade (%)'] = slope

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def process_imu(df):
    mask = df["sensor"] == 'imu'
    df_imu = df[mask]
    
    if df_imu.empty:
        return

    df_imu.rename(columns={2: 'acceleration X (m/s^2)',
                           3: 'acceleration Y (m/s^2)',
                           4: 'acceleration Z (m/s^2)',
                           5: 'Angular Velocity X (rad/s)',
                           6: 'Angular Velocity Y (rad/s)',
                           7: 'Angular Velocity Z (rad/s)',
                          }, inplace=True)

    df_imu.dropna(axis=1, how='all',inplace=True)
    
    df_imu = filter_df_signal(df_imu, 'Angular Velocity X (rad/s)', "gyro_x_filtered", 10 )
    df_imu = filter_df_signal(df_imu, 'acceleration X (m/s^2)', "acceleration_x_filtered", 10 )


    return df_imu

def process_brake(df):
    mask = df["sensor"] == 'Brake State'
    df_brake = df[mask]

    df_brake.rename(columns={2: 'Brake State',
                          }, inplace=True)

    df_brake.dropna(axis=1, how='all',inplace=True)
    return df_brake


def process_pas(df):
    mask = df["sensor"] == 'pas'
    df_pas = df[mask]
    
    if df_pas.empty:
        return


    df_pas.rename(columns={2: 'pulse_delay_us',
                          }, inplace=True)
    df_pas.dropna(axis=1, how='all',inplace=True)
    df_pas = df_pas[df_pas['pulse_delay_us'] > 4000]

    df_pas['Pedal Rotation Speed (RPM)'] = df_pas.apply(lambda x: pulse_width_pas_to_rpm(x['pulse_delay_us']), axis=1)

    df_pas.head()
    
    return df_pas
    
def process_motor_speed(df, df_gps):
    mask = df["sensor"] == 'motor_speed'
    df_ms = df[mask]
    
    
    if df_ms.empty:
        return

    df_ms.rename(columns={2: 'pulse_delay_us',
                          }, inplace=True)
    
    df_ms.dropna(axis=1, how='all',inplace=True)
    
    df_ms = df_ms[df_ms['pulse_delay_us'] > 15000]


    df_ms['Motor Rotation Speed (RPM)'] = df_ms.apply(lambda x: pulse_width_to_rpm(x['pulse_delay_us']), axis=1)
    
    df_ms_merged = pd.merge_asof(df_ms, df_gps, on = 'Datetime (UTC)', direction = 'nearest')
    
    multiplier = df_ms_merged['GPS Horizontal Speed (km/h)'].div(df_ms_merged['Motor Rotation Speed (RPM)'], axis = 0).mean()

    
    
    df_ms['Motor Rotation Speed (RPM)'] = df_ms['Motor Rotation Speed (RPM)'] * multiplier
    
    
    time_delta = df_ms['Datetime (UTC)'].diff().dt.total_seconds().fillna(0)
    df_ms['filtered_motor_rpm'] = signal_filter(df_ms['Motor Rotation Speed (RPM)'], highcut=100, method='butterworth_ba', order=5)
    df_ms['motor_acceleration'] = df_ms["filtered_motor_rpm"].diff()/time_delta


    
    return df_ms

def process_ina(df):    
    mask = df["sensor"] == 'ina226'
    df_ina = df[mask]
    
    SHUNT_RESISTANCE = 0.00215 # ohms

    df_ina.rename(columns={2: 'INA226 ID',
                           3: 'Voltage (V)',
                           4: 'Shunt Voltage Drop (V)',
                           5: 'Current_uncalibrated',
                           6: 'Power_uncalibrated',
                          }, inplace=True)
    df_ina.dropna(axis=1, how='all',inplace=True)
    df_ina.reset_index()


    df_ina = df_ina[df_ina['Voltage (V)'] != 0]
    
    df_ina['Current (mA)'] = df_ina['Shunt Voltage Drop (V)'] / SHUNT_RESISTANCE
    df_ina['Power (mW)'] = df_ina['Current (mA)'] * df_ina['Voltage (V)']

    df_ina["Power_averaged"] = signal_filter(df_ina['Power (mW)'], highcut=6, method='butterworth_ba', order=2)
#     df_ina["Current_averaged"] = signal_filter(df_ina['Current (mA)'], highcut=6, method='butterworth_ba', order=2)
#     df_ina["Voltage_V_averaged"] = signal_filter(df_ina['Voltage (V)'], highcut=6, method='butterworth_ba', order=2)


    print("Total Energy Consumption[KiloJoules]",energy_from_power_time(df_ina['Datetime (UTC)'],df_ina['Power (mW)']))
    df_ina.head()
    
    return df_ina

def process_baro(df,df_ms, df_gps):
    mask = df["sensor"] == 'baro'
    df_baro = df[mask]

    df_baro.rename(columns={2: 'temperature (°C)',
                           3: 'Pressure (mbar)',
                           4: 'humidity (RH%)',
                          }, inplace=True)




    df_baro['Barometric Altitude (m)'] = df_baro.apply(lambda x: get_altitude(x['Pressure (mbar)'], x['temperature (°C)']), axis=1)
    
    df_baro = df_baro[df_baro['Barometric Altitude (m)'] > -30] # Filter out unrealistic pressures
    
    df_baro['Barometric Altitude Filtered (m)'] = signal_filter(df_baro['Barometric Altitude (m)'], highcut=10, method='butterworth_ba', order=2)

    df_baro_merged = pd.merge_asof(df_baro, df_gps, on = 'Datetime (UTC)', direction = 'forward')
    
    
    
    offset = df_baro_merged['Barometric Altitude (m)'].sub(df_baro_merged['GPS altitude (m)'], axis = 0).mean()
    

    time_delta = df_baro['Datetime (UTC)'].diff().dt.total_seconds().fillna(0)
    df_baro['Vertical Rise (m)'] = df_baro['Barometric Altitude Filtered (m)'].diff()
    df_baro['Vertical Velocity (m/s)'] = df_baro['Vertical Rise (m)']/time_delta
    
    df_baro['Barometric_Altitude_Uncalibrated'] = df_baro['Barometric Altitude (m)']
    df_baro['Barometric Altitude (m)'] = df_baro['Barometric Altitude (m)'] - offset
    df_baro['Barometric Altitude Filtered (m)'] = df_baro['Barometric Altitude Filtered (m)'] - offset



    df_baro.dropna(axis=1, how='all',inplace=True)

    df_baro.head()
    
    return df_baro

def process(df):
    print("Start GPS process")
    df_gps = process_gps(df)
    print("GPS process DONE....")
    
    df_pas = process_pas(df)
    print("PAS process DONE....")
    
    df_ms = process_motor_speed(df, df_gps)
    print("Motor Speed process DONE....")

    df_ina = process_ina(df)
    print("INA226 process DONE....")

    df_baro = process_baro(df,df_ms,df_gps)
    print("Barometer process DONE....")
    
    add_slope_to_gps(df_gps, df_baro)
    print("Add slope data DONE....")

    df_imu = process_imu(df)
    print("IMU process DONE....")

    df_brake = process_brake(df)
    print("Brake process DONE....")


    return df_ina, df_gps, df_baro, df_pas, df_ms, df_imu, df_brake


def process_charge_data(fps):
    dfs  = [read_file(fp) for fp in fps]
    df = pd.concat(dfs, ignore_index=True)
    df_ina = process_ina(df)
    return df_ina

def display_charge_data(df_ina):
    
    
    FEATURES = ["Voltage_V_averaged","Current_averaged","Power_averaged"]
    TITLES = ["Battery Voltage[V]","Current[mA]","Power[mW]"]

    N_FEATURES = len(FEATURES)
    fig = make_subplots(rows=N_FEATURES, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.01)

    fig.update_layout(hovermode="x unified")

    for i, feature in enumerate(FEATURES):
        fig.add_trace(go.Scatter(
            x=df_ina.index,
            y=df_ina[feature],
            name=feature,
            hoverinfo='y'),
            row=i+1, col=1)
        
#         non_averaged_feature = feature.replace("_averaged","")
#         fig.add_trace(go.Scatter(
#             x=df_ina.index,
#             y=df_ina[non_averaged_feature],
#             name=non_averaged_feature,
#             hoverinfo='y'),
#             row=i+1, col=1)

        fig.update_yaxes(title_text=TITLES[i], row=i+1, col=1)


    fig.update_layout(title_text="Power Parameters")

    fig.write_html("output/Charging_Power_variables.html")

    fig.show()



params_x_for_ml = ['temperature (°C)', 
                   'Pressure (mbar)', 
                   'humidity (RH%)',
                   'Barometric Altitude Filtered (m)',
                   'Road Grade (%)',
                   'Latitude (°)',
                   'Longitude (°)',
                   'GPS altitude (m)',
                   'heading (radians)',
                   'Distance to nearest road feature (°)',
                   'Road Curvature (m radius)',

                   
#                    'GPS Horizontal Speed (km/h)',
#                    'SOC',
                   
                   
#                     'Vertical Velocity (m/s)',
#                     'Pedal Rotation Speed (RPM)',
#                     'Motor Rotation Speed (RPM)',
#                     'acceleration X (m/s^2)',
#                     'acceleration Y (m/s^2)',
#                     'acceleration Z (m/s^2)',
#                     'Angular Velocity X (rad/s)',
#                     'Angular Velocity Y (rad/s)',
#                     'Angular Velocity Z (rad/s)',

                 ]






# 'Barometric Altitude (m)',
# 'Barometric Altitude Filtered (m)',
# 'Vertical Velocity (m/s)',
# 'Pedal Rotation Speed (RPM)',
# 'Motor Rotation Speed (RPM)',
# 'acceleration X (m/s^2)',
# 'acceleration Y (m/s^2)',
# 'acceleration Z (m/s^2)',
# 'Angular Velocity X (rad/s)',
# 'Angular Velocity Y (rad/s)',
# 'Angular Velocity Z (rad/s)',
# 'Brake State',

params_y_for_ml = 'Power (mW)'

params_x_for_ml_soc = ['temperature (°C)', 'Voltage (V)', 'Current (mA)', 'Power (mW)']
params_y_for_ml_soc = "SOC"



def drop_sensor_column(df, column_name):
    return df[df.columns.difference([column_name])]

    
    
def gen_ml_data(dataframes):
    """
    The first data frame in dataframes should have the highest datarate
    """
    
    column_name = "sensor"
    
    
    print("Dropping Sensor column....")
    for i in dataframes:
        if i is not None:
            if i.empty is False:
                i.drop(column_name, axis=1, inplace=True)

    
        
    print("Merging Dataframes....")

    df_ml = dataframes[0] # dataframe 0 is ina226 data at 10 Hz samplerate.
    
    indexes = [1, 2, 3, 4, 5, 6]
    for index in indexes:
        if dataframes[index] is not None:
            if dataframes[index].empty is False:
                dataframes[index].drop('trip', axis=1, inplace=True)
                df_ml = pd.merge_asof(df_ml, dataframes[index], on = 'Datetime (UTC)', direction = 'forward')
    print("Done merging Dataframes....")


    return df_ml

def split_x_y(df_ml, params_x_for_ml, params_y_for_ml):
    x, y = df_ml[params_x_for_ml], df_ml[params_y_for_ml]
    return x, y

def concat_dfs(fps):
    dfs = []
    
    for fp in fps:
        df = read_file(fp)
        trip_id = secrets.token_hex(3)
        print("Trip id: ", trip_id)

        df["trip"] = trip_id
        
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True)
    return df

def get_raw_dfs(fps):
    df = concat_dfs(fps)
    raw_dfs = process(df)
    
    del df
    gc.collect()

    return raw_dfs
    

def remove_rows_with_na_in_column(df, column):
    return df[df[column].notna()]

def process_for_ml(fps):
    """
    Takes in list of file paths, concats and processes them
    Set display_variables = True to visualise the variables.
    """
    raw_dfs = get_raw_dfs(fps)
    
    # Highest datarate must be in the start of the list
    df_ml = gen_ml_data(raw_dfs)
    #df_ml = remove_rows_with_na_in_column(df_ml, "trip")
    
    
    return df_ml, raw_dfs

def get_energy_error(predicted_energy, actual_energy):
    Error = 100 * (predicted_energy - actual_energy)/actual_energy
    return Error


def print_test_results(xgbr, df_ml):
    x, y = split_x_y(df_ml, params_x_for_ml, params_y_for_ml)
    ypred = xgbr.predict(x)
    
    timestamps = df_ml['Datetime (UTC)']
    print("Test Results :: ")
    error, actual_energy, predicted_energy = print_power_consumption_score(timestamps, y, ypred)
    return error


## SOC Calculations

def coulomb_counting(df):
    """
    Return SOC series, determined using coulomb counting
    """
    total_capacity_As = 8.708 * 3600 # in As
    time_interval =  df['Datetime (UTC)'].diff().dt.total_seconds().fillna(0)
    energy  = (df['Current (mA)']/1000) * time_interval # Convert power from mA to A
    remaining_energy = total_capacity_As - energy.cumsum()
    soc = remaining_energy / total_capacity_As
    return soc

def thevenin_model(df):
    """
    Return SOC series, determined from Current and Voltage only.
    """
    total_capacity_As = 8.708 * 3600 # in As
    time_interval = df['Datetime (UTC)'].diff().dt.total_seconds().fillna(0)
    energy  = (df['Current (mA)']/1000) * time_interval # Convert power from mA to A
    remaining_energy = total_capacity_As - energy.cumsum()
    soc = remaining_energy / total_capacity_As
    return soc

def ML_trained_by_coulomb_counting(df):
    """
    Return SOC series, determined from ML model, trained on Coulomb counting. 
    Uses Voltage, Current, Power and temperature (°C) to determine SOC
    """
    x = df[params_x_for_ml_soc] # WARNING: df_ml_test must be a full discharge of battery from full to empty
    soc = xgbr_SOC.predict(x)
    return soc

def add_soc_feature(df, method):
    for trip in df["trip"].unique():
    
        soc = method(df[df["trip"]==trip])
        
        df.loc[df["trip"]==trip,'SOC'] = soc


    
def score_predicted_data_xgboost(model, df_ml_test, predictor_args=None):
    x, y  = split_x_y(df_ml_test, params_x_for_ml, params_y_for_ml)
    ypred = model.predict(x)
    error, actual_energy, predicted_energy = print_power_consumption_score(df_ml_test['Datetime (UTC)'], y, ypred)
    return y, ypred, error

def make_predictions_xgboost(model, df_ml_test, plot=True, predictor_args=None):
    y, ypred, error = score_predicted_data_xgboost(model, df_ml_test, predictor_args=predictor_args)
    if plot == True:
        plot_predicted_data(df_ml_test['Datetime (UTC)'], y, ypred)
    return error

def plot_3d_plot(df_gps, df_baro, df_ina):
    
    df = pd.merge_asof(df_gps,df_baro , on = 'Datetime (UTC)', direction = 'nearest')
    df = pd.merge_asof(df,df_ina , on = 'Datetime (UTC)', direction = 'nearest')

    fig = go.Figure(data=go.Scatter3d(
        x=df['Longitude (°)'],
        y=df['Latitude (°)'],
        z=df['Barometric Altitude Filtered (m)'],
        marker=dict(
            size=4,
            color=df['Power (mW)'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(
                title='Power (mW)'
            )

        ),
        text = 'Power:' +df['Power (mW)'].astype(str),
        line=dict(
            color='darkblue',
            width=2
        ),
    ))
    
    # Fix long lat scale here: https://stackoverflow.com/a/39540339/13737285
    
    current_latitude = 52 # approximately london

    latitude_km = 111.32
    longitude_km = 40075 * np.cos( np.deg2rad(current_latitude) ) / 360
    

    fig.update_layout(
        width=800,
        #height=700,
        #autosize=False,
        scene=dict(
            xaxis = dict(title='Longitude (°)'),
            yaxis = dict(title='Latitude (°)'),
            zaxis = dict(title='Barometric Altitude Filtered (m)'),
            aspectratio=dict(x=latitude_km/longitude_km, y=1, z=1)
        ),
    )




    fig.write_html("output/gps_speed_3d.html")
    fig.show()

def show_correlations_in_df(df):
    """
    Show correlations between all the columns in df
    """
    fig = px.imshow(df.corr(), title='Heatmap of co-relation between variables')
    fig.write_html("output/Correlation_heatmap.html")
    fig.show()

def remove_values_from_list(lst, value):
    """
    Remove all occurances of value from lst, and return the list minus those values
    """
    return list(filter((value).__ne__, lst))

def plot_linear_correlations(df):
    """
    plot all X-features against output variable Power
    """
    col_= df.columns.tolist()
    col_ = remove_values_from_list(col_, "sensor_y")
    col_ = remove_values_from_list(col_, "sensor_x")

    for i in col_[10:]:
        fig = px.scatter(df, x=i, y="Power", title='{0} vs Power'.format(i))
        fig.write_html("output/Correlation_display_{}_vs_Power.html".format(i))

        fig.show()

def get_masked_items(df, column, items):
    return df[df[column].isin(items)]



def special_test_train_split(df_ml, test_size=0.15, random_state=42):
    """
    return test, train dataframes
    """

    train_frac = 1 - test_size
    l = df_ml["trip"].unique()
    
    
    sz = len(l)
    cut = int(train_frac * sz) #80% of the list
    random.Random(random_state).shuffle(l) # inplace shuffle
    train_trips = l[:cut] # first 80% of shuffled list
    test_trips = l[cut:] # last 20% of shuffled list
    
    print("Unique Trips: ",l, "Train Trips: ",train_trips, "Test Trips: ",test_trips )

    
    return get_masked_items(df_ml, "trip", test_trips), get_masked_items(df_ml, "trip", train_trips)

def do_join_plot(df, x_param, y_param, title):
    p = sns.jointplot(x=x_param,
                  y=y_param,
                  data=df.sample(n=20000, random_state=1),
                  kind="reg",
                  scatter_kws={"s": 1})
    p.fig.suptitle(title)
    #p.ax_joint.collections[0].set_alpha(0)
    p.fig.tight_layout()
    p.fig.subplots_adjust(top=0.95) # Reduce plot to make room 

def do_pair_plot(df, features, title, rotation=0, sample_size=20000):
    g = sns.pairplot(
        df[features].sample(n=sample_size, random_state=1),
        plot_kws={"s": 1}
    )
    
    for ax in g.axes.flatten():
        # rotate x axis labels
        ax.set_xlabel(ax.get_xlabel(), rotation = rotation)
        # rotate y axis labels
        ax.set_ylabel(ax.get_ylabel(), rotation = rotation)
        # set y labels alignment
        ax.yaxis.get_label().set_horizontalalignment('right')

    g.fig.suptitle(
        title,
        y=1.001 # y= some height>1
    ) 
    plt.show()


# Python3 implementation of the approach
from math import sqrt
 
# Function to find the circle on
# which the given three points lie
def findCircle(x1, y1, x2, y2, x3, y3) :
    """
    Return radius and center of circle, given 3 bounding points of circle.
    """    
    
    x12 = x1 - x2;
    x13 = x1 - x3;
 
    y12 = y1 - y2;
    y13 = y1 - y3;
 
    y31 = y3 - y1;
    y21 = y2 - y1;
 
    x31 = x3 - x1;
    x21 = x2 - x1;
 
    # x1^2 - x3^2
    sx13 = np.power(x1, 2) - np.power(x3, 2);
 
    # y1^2 - y3^2
    sy13 = np.power(y1, 2) - np.power(y3, 2);
 
    sx21 = np.power(x2, 2) - np.power(x1, 2);
    sy21 = np.power(y2, 2) - np.power(y1, 2);
    

    f = (sx13 * x12 + sy13 * x12 + sx21 * x13 + sy21 * x13) / (2 * (y31 * x12 - y21 * x13))
    g = (sx13 * y12 + sy13 * y12 + sx21 * y13 + sy21 * y13) / (2 * (x31 * y12 - x21 * y13))
 
    c = -np.power(x1, 2) - np.power(y1, 2) - 2 * g * x1 - 2 * f * y1
 

    # eqn of circle be x^2 + y^2 + 2*g*x + 2*f*y + c = 0
    # where centre is (h = -g, k = -f) and
    # radius r as r^2 = h^2 + k^2 - c
    h = -g;
    k = -f;
    sqr_of_r = h * h + k * k - c;
 
    # r is the radius
    r = np.sqrt(sqr_of_r);
     
    return r,h,k
 

# Do machine learning

## Generate training data

In [ ]:
print("LOADING ROAD FEATURES......")
road_features_geo_df = get_road_features(reload=False) # set to true if it has not been loaded before
print("Done ROAD FEATURES......")

In [ ]:
# No need to run if df_ml has been saved

#!/usr/bin/python
# -*- coding: utf-8 -*-
filepaths = [
#       'hampsted_trip_1-4-2021.csv',
    
#     'data_19-4-21.csv',
#      'data_icah_20-4-21.csv',
#     'data_27-4-21.csv',
#     'data_28-4-21.csv',
#     'data_6-5-21.csv',
#     'data_8-5-2021.csv',
#     'data_10-5-21.csv',
#     'data_15-5-21.csv',
#     "data_17-5-21.csv",
#     'data_18-5-21_enoch.csv',
#     'data_20-5-21_v1.csv',
    
#     'data_20-5-21_v2.csv',
#      'data_20-5-21_v3.csv',
#     'data_21-5-21_v1.csv',
#     'data_21-5-21_v2.csv',
#     'data_25-5-21_v1.csv',
#      'data_25-5-21_v2.csv',
#     'data_25-5-21_v3.csv',
#     'data_25-5-21_v4.csv',
#     'data_25-5-21_v5.csv',
    
#     "data_28-5-21_v1.csv",
#     "data_28-5-21_v2.csv",
#     "data_28-5-21_v3.csv",
#     "data_28-5-21_v4.csv",
#     "data_28-5-21_v5.csv",
#     "data_28-5-21_v6.csv",
#     "data_28-5-21_v7.csv",  
#     "data_29-5-21_v1.csv",
#      "data_29-5-21_v2.csv",
#     "data_29-5-21_v2.csv"
    "data_14-5-21-putney-heath-circuit.csv"


    
]

df_ml, raw_dfs = process_for_ml([raw_data_path+i for i in filepaths])

df_ml.to_pickle("df_ml_putney_loop.pkl")

In [ ]:
df_ml_train = df_ml

In [ ]:
final_table_columns = [
    'Datetime (UTC)',
#     'Voltage (V)',
#     'Current (mA)',
#     'Power (mW)',
    'Latitude (°)',
    'Longitude (°)',
    'GPS altitude (m)',
    'GPS Horizontal Speed (km/h)',
    'vehicle_heading',
    'heading (radians)',
#     'Distance to nearest road feature (°)',
#     'Road Curvature (m radius)',
#     'Road Grade (%)',
#     'temperature (°C)',
#     'Pressure (mbar)',
#     'humidity (RH%)',
    'Barometric Altitude (m)',
    'Barometric Altitude Filtered (m)',
    'Vertical Velocity (m/s)',
#     'Pedal Rotation Speed (RPM)',
#     'Motor Rotation Speed (RPM)',
#     'filtered_motor_rpm',
#     'acceleration X (m/s^2)',
#     'acceleration Y (m/s^2)',
#     'acceleration Z (m/s^2)',
#     'Angular Velocity X (rad/s)',
#     'Angular Velocity Y (rad/s)',
#     'Angular Velocity Z (rad/s)',
#     'Brake State',
    'trip'
]



def prep_pkl(file_path):
    df = pd.read_pickle(file_path)
    
    drop_columns = [col for col in df if col not in final_table_columns]
    
    df.drop(columns=drop_columns, inplace=True)
    return df

pickle_files = [
#     "df_ml_hampsted.pkl",
#     "df_ml_0.pkl",
#     "df_ml_1.pkl",
#     "df_ml_2.pkl",
    "df_ml_putney_loop.pkl"
                ]

dfs = [prep_pkl(i) for i in pickle_files]

df_ml = pd.concat(dfs, ignore_index=True)

In [ ]:
df_ml_test, df_ml_train = special_test_train_split(df_ml, test_size=0.1, random_state=24)
del df_ml
gc.collect()

In [ ]:
df_ml_test.loc[:,"Test-Train"] = "Test"
df_ml_train.loc[:,"Test-Train"] = "Train"

## Key datastats

In [ ]:
max_seconds = 10
def get_key_stats(df):
    
    key_stats = {}
    
    for i in df['trip'].unique():
        trip_df = df[df['trip']==i]
    
        time_delta = trip_df['Datetime (UTC)'].diff().dt.total_seconds().fillna(0)    
        total_ride_time = time_delta[time_delta < max_seconds].sum()/3600
        
        distance = get_horizontal_distance(trip_df)
        distance = distance[distance<100].sum()/1000
        
        key_stats[i] = {"Ride Time (hours)": total_ride_time, "Distance (km)": distance}
        
    key_stats_df = pd.DataFrame.from_dict(key_stats, orient='index')
    
    print("Total Ride Time:{0} hours. Distance: {1} kilometers".format(total_ride_time, distance))
    
    return key_stats_df

key_stats_df = get_key_stats(df_ml_train)


In [ ]:
print

In [ ]:
sns.displot(data=key_stats_df, x="Ride Time (hours)")
key_stats_df


In [ ]:
sns.displot(data=key_stats_df, x='Distance (km)')

In [ ]:
len(df_ml_train)

In [ ]:
len(df_ml_test)

#### Add SOC as a feature

In [ ]:
add_soc_feature(df_ml_train, coulomb_counting)
add_soc_feature(df_ml_test, coulomb_counting)

## Display variables for initial viewing

In [ ]:
display_all_variables(*raw_dfs)

## Display GPS variables

In [ ]:
def display_gps_positions(df_gps, lat_feature='Latitude (°)', long_feature='Longitude (°)'):
    """
    Display GPS positions
    """
    
    df_gps = df_gps.sort_values(by='Datetime (UTC)')
    # Display GPS positions
    fig = px.line_mapbox(df_gps,
                            lat=df_gps[lat_feature],
                            lon=df_gps[long_feature],
                            #color='Test Number',#'Test-Train',#"GPS Horizontal Speed (km/h)",#"trip",#"slope",#"LOCATION Altitude ( m)",,#"Speed(km/h)", # "abs_acceleration" or "gps_acceleration" or "power"
                            zoom=14,
                            hover_data=[#'Datetime (UTC)',
                                        'GPS altitude (m)',
                                        #"sats",
                                        'heading (radians)'
                            
                            ],
                         title="Training and Testing Routes in dataset",
                            #size="LOCATION Accuracy ( m)"
                           )



    fig.update_layout(mapbox_style="carto-positron")
    #fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    fig.write_html("output/GPS_track.html")
    
    fig.show()

In [ ]:

def split_into_n_minute_tests(test_length, row_timestep, df):
    """
    test_length is length of test in seconds
    row_timestep is timestep of each row
    df is the dataframe to split into tests
    """
    
    test_n_rows = test_length/row_timestep
    
    tests = np.divmod(np.arange(len(df)),test_n_rows)[0]+1
    print(tests)

    df.loc[:, 'Test Number'] = tests
    


In [ ]:
split_into_n_minute_tests(600, 0.01, df_ml_test)
df_ml_train.loc[:, 'Test Number'] = 'Training'
df_ml = pd.concat([df_ml_train, df_ml_test])

In [ ]:
display_gps_positions(raw_dfs[1]) # df_gps is index 1 TODO: don't use indexes. use labels for readiblity
# display_gps_positions(df_ml.sample(100000)) # df_gps is index 1 TODO: don't use indexes. use labels for readiblity

### Process and Display Charging data

In [ ]:
# WARNING: Ensure 1970 filter is removed, because the data is not timestamped
df_ina = process_charge_data([raw_data_path+"data_charge_14-5-21.csv"])

In [ ]:
def resample_ina_df(df_ina):
    df_ina = df_ina.set_index('Datetime (UTC)')
    df_ina = df_ina.resample('1S').mean()

    display_charge_data(df_ina)

df_ina_subset = df_ina.head(2500000)
# Display charging profile
resample_ina_df(df_ina_subset)

## View Correlations

In [ ]:
show_correlations_in_df(df_ml_train)

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Generate a large random dataset
rs = np.random.RandomState(33)
d = df_ml_train[df_ml_train.columns.difference(['Barometric Altitude Filtered (m)','filtered_motor_rpm', 'vehicle_heading'])].sample(n=100000)

# Compute the correlation matrix
corr = d.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Show pairplots of data

In [ ]:
%matplotlib qt

features_to_pair_plot = [
    'Voltage (V)',
     'Current (mA)',
   'Power (mW)',
    'SOC',
    #'heading (radians)',
    'Barometric Altitude (m)',
    #'Barometric Altitude Filtered (m)'
    #'Pressure (mbar)',
    "temperature (°C)",
    'Pedal Rotation Speed (RPM)',
    'Motor Rotation Speed (RPM)',
    'Road Grade (%)'
#     "brake_state",
#     "GPS Speed",
#     "altitude",
#     "gps_acceleration"
#     "humidity (RH%)"
]

battery_params = [
    'Voltage (V)',
    'Current (mA)',
    'Power (mW)',
    'SOC',
    'temperature (°C)'
]

In [ ]:
%matplotlib inline
df = df_ml_train
df = df[(df['Road Grade (%)'] > -10000) & (df['Road Grade (%)'] < 10000)]
df = df[(df['Pedal Rotation Speed (RPM)'] > 0) & (df['Pedal Rotation Speed (RPM)'] < 100)]
df = df[(df['Motor Rotation Speed (RPM)'] > 0) & (df['Motor Rotation Speed (RPM)'] < 45)]

do_pair_plot(df, features_to_pair_plot, "Pair Plot of E-bike parameters",sample_size=30001)

In [ ]:
%matplotlib inline

do_pair_plot(df_ml_train, battery_params, "Pair Plot of E-bike Battery parameters", sample_size=40001)

In [ ]:
params_of_interest = [
#     'Current (mA)',
#  'Current_uncalibrated',
#  'Datetime (UTC)',
#  'INA226 ID',
#  'Power (mW)',
#  'Power Averaged (mW)',
#  'Power_uncalibrated',
#  'Shunt Voltage Drop (V)',
#  'Voltage (V)',
 'GPS Horizontal Speed (km/h)',
    'Vertical Velocity (m/s)',
#  'GPS altitude (m)',
#  'fix_type',
#  'gnssFixOK',
#  'GPS Horizontal Acceleration (km/h^2)',
#  'heading (radians)',
#  'hour',
#  'Latitude (°)',
#  'Longitude (°)',
#  'millisecond',
#  'minute',
#  'sats',
#  'second',
#  'Barometric Altitude (m)',
#  'Pressure (mbar)',
#  'Barometric Altitude Filtered (m)',
#  'humidity (RH%)',
 'Road Grade (%)',
#  'temperature (°C)',
#  'Pedal Rotation Speed (RPM)',
#  'pulse_delay_us_x',
#  'filtered_motor_rpm',
#  'motor_acceleration',
#  'Motor Rotation Speed (RPM)',
#  'pulse_delay_us_y',
  'acceleration X (m/s^2)',
#   'acceleration_x_filtered',
#  'acceleration Y (m/s^2)',
#  'acceleration Z (m/s^2)',
#  'Angular Velocity X (rad/s)',
#   'gyro_x_filtered',
  'Angular Velocity Y (rad/s)',
#  'Angular Velocity Z (rad/s)',
#  'Brake State',
]
do_pair_plot(df_ml_train, params_of_interest, "Pair Plot of E-bike parameters")

In [ ]:
params_of_interest = [
#     'Current (mA)',
#  'Current_uncalibrated',
#  'Datetime (UTC)',
#  'INA226 ID',
#  'Power (mW)',
#  'Power Averaged (mW)',
#  'Power_uncalibrated',
#  'Shunt Voltage Drop (V)',
#  'Voltage (V)',
 'GPS Horizontal Speed (km/h)',
    'Vertical Velocity (m/s)',
#  'GPS altitude (m)',
#  'fix_type',
#  'gnssFixOK',
#  'GPS Horizontal Acceleration (km/h^2)',
#  'heading (radians)',
#  'hour',
#  'Latitude (°)',
#  'Longitude (°)',
#  'millisecond',
#  'minute',
#  'sats',
#  'second',
#  'Barometric Altitude (m)',
#  'Pressure (mbar)',
#  'Barometric Altitude Filtered (m)',
#  'humidity (RH%)',
#  'Road Grade (%)',
#  'temperature (°C)',
#  'Pedal Rotation Speed (RPM)',
#  'pulse_delay_us_x',
#  'filtered_motor_rpm',
#  'motor_acceleration',
#  'Motor Rotation Speed (RPM)',
#  'pulse_delay_us_y',
  'acceleration X (m/s^2)',
#   'acceleration_x_filtered',
#  'acceleration Y (m/s^2)',
#  'acceleration Z (m/s^2)',
#  'Angular Velocity X (rad/s)',
#   'gyro_x_filtered',
  'Angular Velocity Y (rad/s)',
#  'Angular Velocity Z (rad/s)',
#  'Brake State',
]

sns.relplot(
    data=df_ml_train.sample(100000),
    s=10,
    x='GPS Horizontal Speed (km/h)', y='Power (mW)',
)

In [ ]:
params_of_interest = [
#     'Current (mA)',
#  'Current_uncalibrated',
#  'Datetime (UTC)',
#  'INA226 ID',
#  'Power (mW)',
#  'Power Averaged (mW)',
#  'Power_uncalibrated',
#  'Shunt Voltage Drop (V)',
#  'Voltage (V)',
 'GPS Horizontal Speed (km/h)',
#  'GPS altitude (m)',
#  'fix_type',
#  'gnssFixOK',
#  'GPS Horizontal Acceleration (km/h^2)',
#  'heading (radians)',
#  'hour',
#  'Latitude (°)',
#  'Longitude (°)',
#  'millisecond',
#  'minute',
#   'sats',
#  'second',
#  'Barometric Altitude (m)',
#  'Pressure (mbar)',
#  'Barometric Altitude Filtered (m)',
#  'humidity (RH%)',
 'Road Grade (%)',
#  'temperature (°C)',
#  'Pedal Rotation Speed (RPM)',
#  'pulse_delay_us_x',
#  'filtered_motor_rpm',
#  'motor_acceleration',
  'Motor Rotation Speed (RPM)',
#  'pulse_delay_us_y',
#   'acceleration X (m/s^2)',
#   'acceleration_x_filtered',
#  'acceleration Y (m/s^2)',
#  'acceleration Z (m/s^2)',
#  'Angular Velocity X (rad/s)',
#   'gyro_x_filtered',
#   'Angular Velocity Y (rad/s)',
#  'Angular Velocity Z (rad/s)',
#  'Brake State',
#   'SOC'
]
do_pair_plot(df_ml_train, params_of_interest, "Comparison of GPS Speed, Motor Internal Speed and GPS Sats")

In [ ]:
params_of_interest = [
#     'Current (mA)',
#  'Current_uncalibrated',
#  'Datetime (UTC)',
#  'INA226 ID',
    'Power (mW)',
#  'Power Averaged (mW)',
#  'Power_uncalibrated',
#  'Shunt Voltage Drop (V)',
#  'Voltage (V)',
  'GPS Horizontal Speed (km/h)',
#  'GPS altitude (m)',
#  'fix_type',
#  'gnssFixOK',
#  'GPS Horizontal Acceleration (km/h^2)',
#  'heading (radians)',
#  'hour',
#  'Latitude (°)',
#  'Longitude (°)',
#  'millisecond',
#  'minute',
#   'sats',
#  'second',
#  'Barometric Altitude (m)',
#  'Pressure (mbar)',
#  'Barometric Altitude Filtered (m)',
#  'humidity (RH%)',
  'Road Grade (%)',
#  'temperature (°C)',
#  'Pedal Rotation Speed (RPM)',
#  'pulse_delay_us_x',
#  'filtered_motor_rpm',
#  'motor_acceleration',
#   'Motor Rotation Speed (RPM)',
#  'pulse_delay_us_y',
#   'acceleration X (m/s^2)',
#   'acceleration_x_filtered',
#  'acceleration Y (m/s^2)',
#  'acceleration Z (m/s^2)',
#  'Angular Velocity X (rad/s)',
#   'gyro_x_filtered',
#   'Angular Velocity Y (rad/s)',
#  'Angular Velocity Z (rad/s)',
#  'Brake State',
#   'SOC'
     'Vertical Velocity (m/s)',
#     "vertical_distance"

]
%matplotlib inline

df = df_ml_train

df = df[(df['Road Grade (%)'] > -100000) & (df['Road Grade (%)'] < 100000)]
df = df[(df['Vertical Velocity (m/s)'] > -0.25) & (df['Vertical Velocity (m/s)'] < 0.25)]
df = df[(df['GPS Horizontal Speed (km/h)'] > -0.5) & (df['GPS Horizontal Speed (km/h)'] < 40)]


do_pair_plot(df, params_of_interest, "Comparison of Gravity parameters and Power", sample_size=20000)

In [ ]:
do_join_plot(df_ml_train, 'Vertical Velocity (m/s)', 'Power (mW)', "vertical_velocity[m/s] vs Power output[mW] from Battery")

In [ ]:
do_join_plot(df_ml_train, "vertical_distance", 'Power (mW)', "vertical_velocity[m/s] vs Power output[mW] from Battery")

In [ ]:
do_join_plot(df_ml_train, "SOC", 'Power (mW)', "State-of-Charge(SOC) vs Power output[mW] from Battery")

In [ ]:
do_join_plot(df_ml, "slope", "vertical_distance", "Vertical distance vs slope")

## Export dataframe to pickle

In [ ]:
def export_to_pickle(df):
    print(df.head())
    df.to_pickle("df_ml_train.pkl")

export_to_pickle(df_ml_train)

### Display SOC vs power

In [ ]:
fig =  px.line(df_ml_train, x='SOC', y='Power (mW)', title='SOC vs Power')
fig.update_xaxes(autorange="reversed")
fig.update_yaxes(title = "Power[mW]")
fig.write_html("output/soc_vs_power.html")
fig.show()

### Display SOC vs Time

In [ ]:
fig =  px.line(df_ml_test, x=df_ml_test['Datetime (UTC)'], y="SOC", title='SOC over Time')
fig.update_yaxes(title = "Power[mW]")
fig.write_html("output/soc_vs_power.html")
fig.show()

### Display barometric altitude vs GPS

In [ ]:
def display_barom_vs_gps(raw_dfs):

    df_ina, df_gps, df_baro, df_pas, df_ms, df_imu, df_brake = raw_dfs

    fig = make_subplots(rows=1, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.01,
                       )


    fig.update_layout(hovermode="x unified")

    fig.add_trace(go.Scatter(
        x=df_baro['Datetime (UTC)'],
        y=df_baro["Barometric_Altitude_Uncalibrated"],
        name="Barometric Altitude Uncalibrated",
        hoverinfo='y'),
        row=1, col=1)

    fig.add_trace(go.Scatter(
        x=df_baro['Datetime (UTC)'],
        y=df_baro["Baro_Altitude"],
        name="Barometric Altitude Calibrated",
        hoverinfo='y'),
        row=1, col=1)

    fig.add_trace(go.Scatter(
        x=df_baro['Datetime (UTC)'],
        y=df_gps["altitude"],
        name="GPS Altitude",
        hoverinfo='y'),
        row=1, col=1)

    fig.update_yaxes(title_text="Altitude[m]", row=1, col=1)
    fig.update_xaxes(title_text="Time", row=1, col=1)


    fig.update_layout(title_text="GPS Altitude vs Barometric Altitude")

    fig.write_html("output/GPS_vs_Baro.html")

    fig.show()

display_barom_vs_gps(raw_dfs)

In [ ]:
def display_ina(raw_dfs):

    df_ina, df_gps, df_baro, df_pas, df_ms, df_imu, df_brake = raw_dfs
    
    #df_ina = df_ina.sample(n=100000).sort_values('Datetime (UTC)')
    df_ina = df_ina.iloc[-100000:]

    fig = make_subplots(rows=1, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.01,
                       )


    fig.update_layout(hovermode="x unified")

    fig.add_trace(go.Scatter(
        x=df_ina['Datetime (UTC)'],
        y=df_ina['Power (mW)'],
        name='Power (mW)',
        hoverinfo='y'),
        row=1, col=1)

    fig.add_trace(go.Scatter(
        x=df_ina['Datetime (UTC)'],
        y=df_ina["Power_averaged"],
        name="Power Averaged",
        hoverinfo='y'),
        row=1, col=1)




    fig.write_html("output/ina.html")

    fig.show()

display_ina(raw_dfs)

## Display 3D plot of trip

In [ ]:
plot_3d_plot(raw_dfs[1], raw_dfs[2], raw_dfs[0])

## Group power into grid squares of longitude/latitude

In [ ]:
def group_data_location(df):
    step = 0.0002
    to_bin = lambda x: np.floor(x / step) * step
    df['Latitude (°)'] = df['Latitude (°)'].map(to_bin)
    df['Longitude (°)'] = df['Longitude (°)'].map(to_bin)
    
    
#     step = 0.1
#     to_bin = lambda x: np.floor(x / step) * step
#     df["headingbin"] = df['heading (radians)'].map(to_bin)
    
    groups = df.groupby([
        "Latitude (°)", 
        'Longitude (°)',
#         'headingbin'
    ]).mean().reset_index()
    
    
    
    return groups

def group_data_time_interval(df):
    groups = df.groupby(pd.Grouper(key='Datetime (UTC)', freq="1s")).mean()
    #df['Datetime (UTC)'] = df.index
    groups = groups.dropna()
    return groups

# df_ml_train_grouped = group_data_time_interval(df_ml_train)
# display_gps_positions_bins(df_ml_train_grouped)
df_ml_test_grouped = group_data_location(df_ml_test)

In [ ]:
df_ml_test_grouped

In [ ]:
display_gps_positions(df_ml_test_grouped, lat_feature='Latitude (°)', long_feature='Longitude (°)')

## Display distribution of power parameters

In [ ]:
import plotly.express as px
fig = px.histogram(raw_dfs_train[0], x='Power (mW)', title='Power Distribution')
fig.update_xaxes(title_text="Power[mW]")
fig.show()
fig = px.histogram(raw_dfs_train[0], x='Voltage (V)', title='Battery Voltage Distribution')
fig.update_xaxes(title_text="Voltage[V]")
fig.show()
fig = px.histogram(raw_dfs_train[0], x="Current", title='Current Distribution')
fig.update_xaxes(title_text="Current[mA]")

fig.show()

### Display speed vs power

In [ ]:
fig =  px.line(raw_dfs_train[4], x='Datetime (UTC)', y="filtered_motor_rpm", title='Speed vs Time').show()
fig =  px.line(raw_dfs_train[4], x='Datetime (UTC)', y="motor_rpm", title='Speed vs Time').show()

### Display Lat/Long vs power(unbinned)

In [ ]:
%matplotlib qt

#latitude_start_pt, longitude_start_pt = 51.45282, -0.2275045
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px


L1 = [51.45282, -0.2275045]
def plot_proximity_to_start_point(df):
    df['distance'] = df[['Latitude (°)', 'Longitude (°)']].sub(np.array(L1)).pow(2).sum(1).pow(0.5)
    
    fig = px.line(df, x='Datetime (UTC)', y="distance", title='Distance from start point').show()
    
    time_series = df['distance']
    indices = find_peaks(-time_series, distance = 2000,height=-0.0005)[0]
    
    
    df['Loop number'] = 0
    for i in range(len(indices)-1):
        rows = range(indices[i],indices[i+1])
        
        df.loc[rows, 'Loop number'] = i+1
    


    
plot_proximity_to_start_point(df_ml_train)




In [ ]:
def plot_distribution_on_each_loop(df):
    df = df.sample(frac=0.1)
    unique_loops = sorted(df['Loop number'].unique())
    # Group data together
    hist_data = [df[df['Loop number']==i]['Power (mW)'] for i in unique_loops]

    group_labels = ["Lap "+str(i) for i in unique_loops]

    # Create distplot with custom bin_size
    fig = ff.create_distplot(hist_data, group_labels, bin_size=10000)
    
    fig.update_layout(
        title="Distribution of Power values, over the course of 12 laps",
    )
    
    
    fig.update_xaxes(title="Power (mW)")
    fig.update_yaxes(title="Fraction of power values")
    fig.write_html("output/Distribution-of-power-values-each-lap.html")

    fig.show()

plot_distribution_on_each_loop(df_ml_train)


In [ ]:
from sklearn.neighbors import KDTree
import numpy as np

def plot_params_on_each_loop(df):
    
    
    df['Power (mW) Filtered'] =  df.rolling(window=100)['Power (mW)'].mean().fillna(0)
    
    
    target_loop_n = 3
    

    X = df[df['Loop number']==target_loop_n][['Latitude (°)', 'Longitude (°)']]
    kdt = KDTree(X, leaf_size=30, metric='euclidean')
    indexes = kdt.query(df[['Latitude (°)', 'Longitude (°)']], k=1, return_distance=False)
    
    df["Trip Progress (%)"] = 100 * (indexes - indexes.min())/(indexes.max() - indexes.min())
    
    
    df_list = []
    for i in df['Loop number'].unique():
        df_list.append(df[df['Loop number'] == i].drop_duplicates(subset=['Trip Progress (%)']))
   
    df = pd.concat(df_list, ignore_index=True)
    
    df.sort_values(by=['Loop number','Trip Progress (%)'], inplace=True)


    fig = go.Figure()

    
    for i in df['Loop number'].unique():        
        fig.add_trace(go.Scatter(x=df[df['Loop number'] == i]['Trip Progress (%)'],
                                 y=df[df['Loop number'] == i]['Power (mW) Filtered'],
                                 fill='tozeroy',
                                 name = 'Lap {0}'.format(i)
                                )
                     ) # fill down to xaxis

    fig.update_layout(
        title="Power profile(low pass filtered) on each loop",
    )
    
    fig.update_xaxes(title="Lap Progress (%)")
    fig.update_yaxes(title="Power (mW)")


    
    fig.write_html("output/Power_profile_on_each_loop.html")

    fig.show()
    


plot_params_on_each_loop(df_ml_train.dropna().sort_values(by='Datetime (UTC)'))


### Calculate Energy Consumption each loop

In [ ]:
def calculate_energy_consumption_for_each_loop(df_ml):
    energies_per_loop = []
    for i in df_ml['Loop number'].unique():
        
        df_loop = df_ml[df_ml["Loop number"] == i]
        energy_in_loop = energy_from_power_time(df_loop['Datetime (UTC)'], df_loop['Power (mW)'])
        energies_per_loop.append((i,energy_in_loop))
                                 
    return pd.DataFrame(energies_per_loop, columns=['Loop number', 'Energy Per Loop (kJ)'])

def plot_energy_consumption_per_loop(df_ml):
    energies_per_loop = calculate_energy_consumption_for_each_loop(df_ml)       
    fig = px.line(energies_per_loop, x='Loop number', y='Energy Per Loop (kJ)', title='Energy consumption of each loop around Putney Heath')
    fig.update_layout(yaxis_range=[48,65])
    
    fig.update_layout(
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 1,
            dtick = 1
        )
    )
    fig.write_html("output/Energy_consumption_per_loop.html")
    fig.show()

plot_energy_consumption_per_loop(df_ml_train)

### Display Lat/Long vs power(binned)

In [ ]:
def plot_proximity_to_start_point_binned(df):
    df['distance'] = df[['Latitude (°)', 'Longitude (°)']].sub(np.array(L1)).pow(2).sum(1).pow(0.5)
    
    
    time_series = df['distance']
    indices = find_peaks(-time_series,
                         distance = 2000,
                         height=-0.0005)[0]
    
    
    df["ts"] = df.index.values
    df['Loop number'] = 0
    for i in range(len(indices)-1):
        
        start_time = df["ts"].iloc[indices[i]]
        end_time = df["ts"].iloc[indices[i+1]]        
        df.loc[start_time:end_time, 'Loop number'] = i+1
    
    fig = px.line(df, x='Latitude (°)', y='Power (mW)', color='Loop number', title= "Power profile on each loop")
    fig.write_html("output/Power_profile_on_each_loop_binned.html")

    fig.show()

plot_proximity_to_start_point_binned(df_ml_train)

In [ ]:
fig =  px.scatter(df_ml_train, x="Latitude (°)", y="Power", title='Latitude vs Power').show()

### Display IMU data

In [ ]:

def display_imu_plots(df_imu):
    fig = go.Figure()
    
    # Add traces
    fig.add_trace(go.Scatter(x=df_imu["gyro_x"],
                             y=df_imu["acceleration_x"],
                             mode='markers',
                             marker=dict(size=1),
                             name='Raw data'
                            )
                 )
    
    fig.add_trace(go.Scatter(x=df_imu["gyro_x_filtered"], 
                             y=df_imu["acceleration_x_filtered"],
                             mode='markers',
                             marker=dict(size=1),
                             name='Filtered data'

                            )
                 )
    
    fig.update_layout(
        title="IMU Acceleration vs Angular velocity(gyro)",
        xaxis_title="Angular Velocity[rad/s]",
        yaxis_title="Acceleration[m/s^2]",
    )
    fig.show()
                  
display_imu_plots(raw_dfs_train[5])

In [ ]:
fig = px.scatter(raw_dfs_train[5], x="gyro_x", y="acceleration_x", title='Acceleration[m/s^2] vs Angular velocity[rad/s]')
fig.update_traces(marker=dict(size=1))
fig.show()

In [ ]:
fig = px.scatter(raw_dfs_train[5], x='Datetime (UTC)', y=["acceleration_x","acceleration_x_filtered"], title='Acceleration over Time').show()

In [ ]:
fig = px.line(df_ml_train, x='Datetime (UTC)', y=["Power","Power_averaged"], title='Power over Time').show()

## Train ML model XGboost

In [ ]:
def plot_predicted_data(timestamps, ytest, ypred, scores, show_html=False, save_image=True):
    
    fig = go.Figure()
    fig.add_traces(go.Scatter(x=timestamps, y=ytest, name='Actual data'))
    fig.add_traces(go.Scatter(x=timestamps, y=ypred, name='Regression Fit'))
    
    title = "Power consumption, Actual and Predicted.<br>Predicted Energy consumption:{0:.2f} kJ, Actual Energy consumption:{1:.2f} kJ,<br>Trip Prediction Error:{2:.1f} % RMSE Score:{3:.0f} W".format(scores["predicted"],
                                                                                                                                              scores["actual"],
                                                                                                                                              scores["percentage_error"],
                                                                                                                                              scores["RMSE score"]/1000

                                                                                                                                             )

    fig.update_layout(
        title=title,
        xaxis_title="Time(UTC)",
        yaxis_title="Power[mW]",
    )
    fig.write_html("output/Predicted_plot.html")
    
    if save_image:
        ts = timestamps.iloc[0]
        print(ts)
        fig.write_image("output/images/predictions_on_{0}.jpeg".format(ts),scale=4, width=1600, height=900)

        
    if show_html:
        fig.show()
        
def split_into_test_runs(test_length, row_timestep, df_ml_test):
    test_n_rows = test_length/row_timestep

    n_rows = len(df_ml_test.index)

    tests = np.array_split(df_ml_test, n_rows//test_n_rows)
    
    return tests

def test_short_runs(xgbr, df_ml_test, predictor, scaler = None, test_length = 600, row_timestep = 0.01, show_plots=False, predictor_args=None):

    tests = split_into_test_runs(test_length, row_timestep, df_ml_test);
    
    scores = []
    for test in tests:
        error = predictor(xgbr, test, plot=show_plots, predictor_args=predictor_args)
        scores.append(error)
        
    return pd.Series(scores)


def print_power_consumption_score(timestamps, ytest, ypred):
    actual_energy = energy_from_power_time(timestamps, ytest)
    predicted_energy = energy_from_power_time(timestamps, ypred)
    error = get_energy_error(predicted_energy, actual_energy)
    
    return error, actual_energy, predicted_energy

def score_predicted_data(model, df_ml_test, scalar=None, predictor_args=None):
    x, y  = split_x_y(df_ml_test, params_x_for_ml, params_y_for_ml)
    
    if scalar:
        x = scaler.transform(x)
        

    if predictor_args:
        ypred = model.predict(x, *predictor_args).flatten()
    else:
        ypred = model.predict(x).flatten()
        
    ypred=ypred.clip(min=0)

    
    rmse = mean_squared_error(y, ypred, squared=False)
    
    a = y - ypred
    abs_error = a.abs()
    
    error, actual_energy, predicted_energy = print_power_consumption_score(df_ml_test['Datetime (UTC)'], y, ypred)
    
    print("Actual energy:",actual_energy, "Predicted Energy:", predicted_energy, "Error[%](ideal should be 0%):", error, "% ", "RMSE Score: {:,}".format(rmse) )

    return y, ypred, error, actual_energy, predicted_energy, rmse, abs_error

def make_predictions(model, df_ml_test, scalar=None, plot=True, predictor_args=None):
    y, ypred, error, actual_energy, predicted_energy, rmse, abs_error = score_predicted_data(model, df_ml_test, scalar=scalar, predictor_args=predictor_args)
    
    scores = {"predicted":predicted_energy,"actual":actual_energy,"percentage_error":error, "RMSE score": rmse}
    if plot == True:
        plot_predicted_data(df_ml_test['Datetime (UTC)'], y, ypred, scores)
    return error

# USE XGBOOST predictor

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error



def do_ml(x,y):
    parmas = {'max_depth': 12, 
              'learning_rate': 0.04823939007347505, 
              'colsample_bytree': 0.2577650725393863,
              'subsample': 0.4278423362185552, 
              'alpha': 0.05460670834513764, 
              'lambda': 0.01647475526883333,
              'min_child_weight': 24.82354496640231,
              'verbosity':2,
              'tree_method':'gpu_hist',
              'gpu_id':0,
              'n_estimators':200
             }

    xgbr = xgb.XGBRegressor(**parmas)
    print(xgbr)
    
    xgbr.fit(x, y)
    

    print("Training score R2: ", xgbr.score(x, y))
    

    _ = plot_importance(xgbr, height=0.9)
    
    return xgbr


In [ ]:
%matplotlib inline

X, y = split_x_y(df_ml_train, params_x_for_ml, params_y_for_ml)

xgbr = do_ml(X, y)
xgbr.save_model('xgbr_model_offline_only_hyper_optimised.json')

In [ ]:
xgbr = xgb.XGBRegressor()
xgbr.load_model('xgbr_model_offline_only.json')

## Create test of 15 minute runs

In [ ]:
frac = 0.1

In [ ]:
scores = test_short_runs(xgbr, df_ml_test.sample(frac=frac).sort_values(by='Datetime (UTC)'), make_predictions, test_length = 60 * 30, row_timestep = 0.01/frac, show_plots=True)

In [ ]:
scores = scores[(scores > -1000) & (scores < 1000)] 

sns.displot(x=scores, kde=True)
pd.DataFrame(scores).describe()

In [ ]:
def get_scores_for_range_of_trip_lengths(time_lengths):
    
    scores_list = []

    for duration in time_lengths:
        scores = test_short_runs(xgbr, 
                                 df_ml_test.sample(frac=frac).sort_values(by='Datetime (UTC)'),
                                 make_predictions,
                                 test_length = 60 * duration, 
                                 row_timestep = 0.01/frac,
                                 show_plots=False)
        
        scores = scores[(scores > -1000) & (scores < 1000)] 

        for i in scores:
            scores_list.append({"Trip Duration (minutes)": duration, "Trip Error (%)": i })
            
        
    return pd.DataFrame(scores_list)


scores_df = get_scores_for_range_of_trip_lengths(range(5,30,5))



In [ ]:
scores_df

In [ ]:
sns.set_theme()

sns.displot(scores_df,
            x="Trip Error (%)",
            hue="Trip Duration (minutes)",
            stat="density",
            common_norm=False,
            palette="tab10",
#             kind="kde", 
#             multiple="stack"
           )


## Plot RMSE under different conditions

In [ ]:
from sklearn.metrics import mean_squared_error


y, ypred, error, actual_energy, predicted_energy, rmse, abs_error= score_predicted_data(xgbr, df_ml_test, scalar=None, predictor_args=None)

In [ ]:
df_ml_test.loc[:,'Absolute Error Power Prediction (mW)'] = abs_error

In [ ]:
df_ml_test

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")

g = sns.jointplot(x="SOC", y='Absolute Error Power Prediction (mW)', data=df_ml_test.sample(10000),
                  kind="reg",
                  truncate=False,
                  color="m",
                  height=7,
                  scatter_kws={'s': 2}
                 )

In [ ]:
g = sns.jointplot(x='GPS Horizontal Speed (km/h)', y='Absolute Error Power Prediction (mW)', data=df_ml_test.sample(10000),
                  kind="reg",
                  truncate=False,
                  color="m",
                  height=7,
                  scatter_kws={'s': 2}
                 )

## Hyperparameter optimise with Optuna


In [ ]:
import os
import string

import numpy as np
import pandas as pd
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from optuna import create_study
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback

FS = (14, 6)  # figure size
RS = 124  # random state
N_JOBS = 8  # number of parallel threads

# repeated K-folds
N_SPLITS = 10
N_REPEATS = 1

# Optuna
N_TRIALS = 100
MULTIVARIATE = True

# XGBoost
EARLY_STOPPING_ROUNDS = 100

In [ ]:
X, y = split_x_y(df_ml_train, params_x_for_ml, params_y_for_ml)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
def objective(trial, X, y, random_state=22, n_splits=3, n_repeats=2, n_jobs=1, early_stopping_rounds=50,):
    # XGBoost parameters
    params = {
        "tree_method":'gpu_hist',
        "gpu_id": 0,
        "verbosity": 2,  # 0 (silent) - 3 (debug)
        "objective": "reg:squarederror",
        "n_estimators": 500,
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": random_state,
        "n_jobs": n_jobs,
    }

    model = XGBRegressor(**params)
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-rmse")
    rkf = RepeatedKFold(
        n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
    )
    X_values = X.values
    y_values = y.values
    y_pred = np.zeros_like(y_values)
    for train_index, test_index in rkf.split(X_values):
        X_A, X_B = X_values[train_index, :], X_values[test_index, :]
        y_A, y_B = y_values[train_index], y_values[test_index]
        model.fit(
            X_A,
            y_A,
            eval_set=[(X_B, y_B)],
            eval_metric="rmse",
            verbose=0,
            callbacks=[pruning_callback],
            early_stopping_rounds=early_stopping_rounds,
        )
        y_pred[test_index] += model.predict(X_B)
    y_pred /= n_repeats
    return np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
sampler = TPESampler(seed=RS, multivariate=MULTIVARIATE)
study = create_study(direction="minimize", sampler=sampler)
study.optimize(
    lambda trial: objective(
        trial,
        X_train,
        y_train,
        random_state=RS,
        n_splits=N_SPLITS,
        n_repeats=N_REPEATS,
        n_jobs=8,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
    ),
    n_trials=N_TRIALS,
    n_jobs=1,
)

# display params
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

## Display interesting variables

In [ ]:
display_interesting_variables(df_ml_test, xgbr)

## Use Neural Network with Structured data input to do regression
Currently, we pass rows into the XG boost model. What if we could insert a snap shot of 10 seconds of data containing all the features, and calculating the energy consumption of this snapshot? Its like a photograph used in Deep Neural Networks: 2 dimensional input. Run the StructuredDataRegressor.
You can also leave the epochs unspecified for an adaptive number of epochs.

In [ ]:
# Initialize the structured data regressor.
reg = ak.StructuredDataRegressor(
    #overwrite=True,
    max_trials=1
)  # It tries 3 different models.



df_ml_train.dropna(subset=params_x_for_ml, inplace=True)
x, y = split_x_y(df_ml_train, params_x_for_ml, params_y_for_ml)

scaler = StandardScaler()

x = scaler.fit_transform(x)


# Feed the structured data regressor with training data.
with tf.device('/gpu:0'):
    reg.fit(x,
            y,
            epochs=2
           )

model = reg.export_model()

print(type(model))  # <class 'tensorflow.python.keras.engine.training.Model'>

try:
    model.save("model_autokeras", save_format="tf")
except Exception:
    model.save("model_autokeras.h5")


In [ ]:
def plot_predicted_data_autokeras(model, df_ml_test,scaler):
    x, y  = split_x_y(df_ml_test, params_x_for_ml, params_y_for_ml)
    x = scaler.transform(x)
    ypred = model.predict(x).flatten()    

    print_power_consumption_score(df_ml_test['Datetime (UTC)'], y, ypred)
    plot_predicted_data(df_ml_test['Datetime (UTC)'], y, ypred)

In [ ]:
plot_predicted_data_autokeras(reg, df_ml_test.loc[:1e6], scaler)

In [ ]:
def score_predicted_data(model, df_ml_test, scalar=None):
        
    x, y  = split_x_y(df_ml_test, params_x_for_ml, params_y_for_ml)
    
    if scalar:
        x = scaler.transform(x)
        
    ypred = model.predict(x).flatten()
    r2_score = model.score(y, ypred)
    error, actual_energy, predicted_energy = print_power_consumption_score(df_ml_test['Datetime (UTC)'], y, ypred)
    return y, ypred, error

def make_predictions_keras_structured(model, df_ml_test, scalar=None, plot=True):
    y, ypred, error, r2_score = score_predicted_data(model, df_ml_test, scalar)
    if plot == True:
        plot_predicted_data(df_ml_test['Datetime (UTC)'], y, ypred, r2_score)
    return error

In [ ]:
scores = test_short_runs(reg, df_ml_test, make_predictions_keras_structured, test_length = 60 * 10, row_timestep = 0.01, show_plots=False)

In [ ]:
sns.displot(x=scores, kde=True)
pd.DataFrame(scores).describe()

## Use Scikit learn simple functions

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import svm


scaler = StandardScaler()

df_ml_train.dropna(subset=params_x_for_ml, inplace=True)
x, y = split_x_y(df_ml_train, params_x_for_ml, params_y_for_ml)
x = scaler.fit_transform(x)


#reg = svm.SVR().fit(x, y)
reg = LinearRegression().fit(x, y)
print("Score: ",reg.score(x, y))

In [ ]:
plot_predicted_data_autokeras(reg, df_ml_test, scaler)

In [ ]:

def score_predicted_data_sk(model, df_ml_test):
    x, y  = split_x_y(df_ml_test, params_x_for_ml, params_y_for_ml)
    x = scaler.transform(x)
    ypred = model.predict(x)
    error, actual_energy, predicted_energy = print_power_consumption_score(df_ml_test['Datetime (UTC)'], y, ypred)
    return y, ypred, error

def make_predictions_sk(model, df_ml_test, plot=True):
    y, ypred, error = score_predicted_data_sk(model, df_ml_test)
    if plot == True:
        plot_predicted_data(df_ml_test['Datetime (UTC)'], y, ypred)
    return error

In [ ]:
scores = test_short_runs(reg, df_ml_test, make_predictions_sk, test_length = 5*60, row_timestep = 0.01)

In [ ]:
scores = np.clip(scores, -100, 100 )
sns.displot(x=scores, kde=True)

## Use Light BGM

In [ ]:
%matplotlib inline
import optuna
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import lightgbm as lgb
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, train_test_split


In [ ]:
def objective(trial, data=df_ml_train[params_x_for_ml], target=df_ml_train[params_y_for_ml]):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param = {
        #'metric': ['l2', 'auc'],
        'random_state': 48,
        'n_estimators': 100,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'min_data_per_group' : trial.suggest_int('min_data_per_group', 1, 100),
        'verbose': 0,


    }
    model = LGBMRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.trials_dataframe()

In [ ]:
#plot_optimization_histor: shows the scores from all trials as well as the best score so far at each point.
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
went and which parts of the space were explored more.

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
#plot_contour: plots parameter interactions on an interactive chart. You can choose which hyperparameters you would like to explore.
optuna.visualization.plot_contour(study, params=['num_leaves',
                            'max_depth',
                            'subsample',
                            'learning_rate',
                            'subsample'])

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
#Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [ ]:
from sklearn.model_selection import cross_val_score

gbm = LGBMRegressor(**study.best_params)
scores = cross_val_score(gbm, X_train, y_train, cv=3)
scores

## Run single training

In [ ]:
hyper_params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     #'metric': ['l2', 'auc'],
#     'learning_rate': 0.005,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.7,
#     'bagging_freq': 10,
#     'verbose': 0,
#     "max_depth": 8,
#     "num_leaves": 128,  
#     "max_bin": 512,
#     "num_iterations": 100000,
    "n_estimators": 1000
}

params = study.best_params
params["n_estimators"] = 1000

gbm = lgb.LGBMRegressor(**hyper_params)
#gbm = lgb.LGBMRegressor(**params)

print(gbm)


gbm.fit(*split_x_y(df_ml_train, params_x_for_ml, params_y_for_ml),
        eval_set=[split_x_y(df_ml_test, params_x_for_ml, params_y_for_ml)],
       )



print("Training score R2: ", gbm.score(*split_x_y(df_ml_train, params_x_for_ml, params_y_for_ml)))

lgb.plot_importance(gbm, height=0.9)

In [ ]:
scores = test_short_runs(gbm, df_ml_test, make_predictions, test_length = 60 * 10, row_timestep = 0.01, show_plots=False, predictor_args=gbm.best_iteration_)

In [ ]:
scores = scores[(scores > -1000) & (scores < 1000)] 


sns.displot(x=scores, kde=True)
pd.DataFrame(scores).describe()

## Now try to use the ImageRegression for autokeras

To make this tutorial easy to follow, we just treat MNIST dataset as a
regression dataset. It means we will treat prediction targets of MNIST dataset,
which are integers ranging from 0 to 9 as numerical values, so that they can be
directly used as the regression targets.


In [ ]:
def prep_for_autokeras_image_regressor(df_ml):
    
    chunks = np.array_split(
        df_ml[params_x_for_ml+[params_y_for_ml]].to_numpy(),
        range(0, len(df_ml), 100) # 100 x 0.01 seconds chunks = 1 second chunks
    )

    chunks = chunks[1:-1] # Drop the first and last chunk that may be shorter

    energies = []
    chunks_edited = []

    time_interval = 0.010 # seconds
    for chunk in chunks:
        powers = chunk[:, -1] # for last column in mW
        energy = np.sum(powers * time_interval / 1000000) # in KiloJoules
        chunks_edited.append(chunk[:, :-1]) # for all but last column
        energies.append(energy)

    
    fig =  px.line(y=energies, title='Energies islotated').show()



    y = np.array(energies)
    x = np.array(chunks_edited)

    print(x.shape)
    print(y.shape)
    
    return x, y

In [ ]:
def do_ml_image_regression(x, y):
    #X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=42)

    # Initialize the image regressor.
    reg = ak.ImageRegressor(overwrite=True,
                            max_trials=3
                           )
    # Feed the image regressor with training data.
    reg.fit(x, y)

    # Evaluate the best model with testing data.
    #print(reg.evaluate(X_test, y_test))
    
    return reg

reg  = do_ml_image_regression(*prep_for_autokeras_image_regressor(df_ml_train))

In [ ]:
# Predict with the best model.

def predict_and_score(reg, df_ml_test):
    x, y  = prep_for_autokeras_image_regressor(df_ml_test)
    predicted_y = reg.predict(x)

    fig =  px.line(y=predicted_y.flatten().tolist(), title='predicted_y').show()


    length = len(y)

    # intialise data of lists.
    data = {'Energies':predicted_y.flatten().tolist() + y.tolist(),
            'type_of_data':length * ["Predicted"] + length * ["Actual"],
            "Index": list(range(length))*2}

    # Create DataFrame
    df = pd.DataFrame(data)

    # Print the output.
    fig = px.line(df, x = "Index", y="Energies", color="type_of_data", title='Energies').show()

    get_energy_error(sum(predicted_y), sum(y))
    
predict_and_score(reg, df_ml_test)